In [1]:
import pandas as pd
import numpy as np
import re

In [2]:
import pandas as pd
import numpy as np
import re

test = pd.read_csv('../source/test', sep='\t')
train = pd.read_csv('../source/train', nrows=20000, sep='\t')
droplist = []
for i, row in train.iterrows():
    if type(row['tokens']) != type(''):
        print(i)
        droplist.append(i)
train = train.drop(droplist)
len_train = len(train)
trtest = pd.concat((train, test), ignore_index=True)
trtest

819
1301
1818
1906
2332
2860
2927
2998
4434
4546
4665
4741
5400
6154
6233
6319
7061
7081
8245
8682
8897
9046
9781
10411
12589
12684
12754
13018
13793
13917
15416
16081
16637
16642
17168
19799


,CLIENT_ID,RETRO_DT,tokens,DEF,urls_hashed
0,5909886,20200911,форм 1 мачеха 1 работы 1 пасынка 1 горячую 1 м...,0.0,7bbcc146c0ba4df814701ff7297b2205 1 ee0d49cbc99...
1,1385448,20210519,platiza 1 мобильные 2 sergej 1 code 3 отказов ...,1.0,1889e0cbd3e343e9521a07aa312257ac 2 8253d8d3a67...
2,1866195,20210913,sims 3 кредит 1 языке 1 разлива 1 огромное 1 о...,0.0,6e617b4bfdbef85d8b09ec08ac72f0f2 1 94309f5f116...
3,1587081,20210702,code 1 историй 1 scf 1 шаг 1 деньги 3 серый 1 ...,0.0,1e833434273e04ba76cfcfb4b48ad21b 3 aee71c8d18d...
4,1400908,20210522,отказов 2 tovara 1 сергея 1 купальный 1 чёрный...,0.0,83e85bee0d81c0e9e5047e202cddf534 3 6e617b4bfdb...
...,...,...,...,...,...
174763,820037931,20220408,материнские 1 hats 1 чёрный 1 деньги 14 бытово...,NaN,1889e0cbd3e343e9521a07aa312257ac 1 aee71c8d18d...
174764,820043321,20220408,кредит 8 служба 2 шаг 2 наличными 10 зубастики...,NaN,68d22a2c8d2fa49d80bbfe9edd8845f0 2 69adb4ce9c4...
174765,820058971,20220409,циан 1 novogodnie 2 результат 1 центр 1 платье...,NaN,a51fa12f9f44dde93d7e36e4ae1c4a0f 1 a18bcf54adc...
174766,820062491,20220409,макаронных 1 mny 1 снять 1 svete 1 платье 1 mn...,NaN,aee71c8d18d6b91b36bf599208a91777 122 d3862674d...


In [3]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

corpus = trtest['tokens']
for i in range(len(corpus)):
    if type(corpus[i]) == type(''):
        corpus[i] = re.sub("\d+", "", corpus[i])
    else:
        corpus[i] = ''
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df=30, max_df=90)
X2 = vectorizer2.fit_transform(corpus)
X2.toarray()[:len(train), :].shape

/tmp/ipykernel_1175/2222497545.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus[i] = re.sub("\d+", "", corpus[i])
/tmp/ipykernel_1175/2222497545.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  corpus[i] = ''


(19964, 19003)

In [24]:
np.save('X2.npy', X2, allow_pickle=True)

In [25]:
X2_copy = np.load('X2.npy', allow_pickle=True)
X2_copy.shape

()

In [4]:
X_train = pd.DataFrame(X2.toarray()[:len(train), :], columns = np.arange(X2.toarray().shape[1]))
X_test = pd.DataFrame(X2.toarray()[len(train):, :], columns = np.arange(X2.toarray().shape[1]))

In [5]:
model = LogisticRegression()
model.fit(X_train, train['DEF'])
answer = pd.DataFrame(np.zeros(len(test)), columns=['DEF'])
for i, text in enumerate(test['tokens']):
    if type(row['tokens']) == type(''):
        answer['DEF'][i:i+1] = model.predict(X_test[i:i+1])
    else:
        answer['DEF'][i:i+1] = 0
answer.head(11)

/home/jupyter-admin/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(

KeyboardInterrupt



In [ ]:
answer.to_csv('ans.csv')

In [11]:
len([answer['DEF'][i] for i in range(len(answer)) if answer['DEF'][i]])

83

In [ ]:
#from xgboost import XGBClassifier
from sklearn.ensemble import BaggingClassifier

model = BaggingClassifier()
model.fit(X_train, train['DEF'])
answer = pd.DataFrame(np.zeros(len(test)), columns=['DEF'])
for i, text in enumerate(test['tokens']):
    if type(row['tokens']) == type(''):
        answer['DEF'][i:i+1] = model.predict(X_test[i:i+1])
    else:
        answer['DEF'][i:i+1] = 0
answer.head(11)

In [3]:
!pip install xgboost

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 255.9 MB 67 kB/s              


# Validation

In [12]:
import pandas as pd
import numpy as np
import re

#test = pd.read_csv('../source/test', nrows=1000, sep='\t')
train = pd.read_csv('../source/train', sep='\t')
droplist = []
for i, row in train.iterrows():
    if type(row['tokens']) != type(''):
        print(i)
        droplist.append(i)
train = train.drop(droplist)
len_train = len(train) // 1.25

819
1301
1818
1906
2332
2860
2927
2998
4434
4546
4665
4741
5400
6154
6233
6319
7061
7081
8245
8682
8897
9046
9781
10411
12589
12684
12754
13018
13793
13917
15416
16081
16637
16642
17168
19799
20328
20467
20662
21984
22075
22323
22727
22991
23881
24392
24888
25646
26392
27542
27783
29094
30049
30671
31000
31658
32330
32811
33136
33613
34293
34377
34920
36289
38472
40226
41170
41459
41974
42034
42859
43070
43495
44096
44447
44879
45195
45196
45343
45408
46905
47837
50245
50275
50430
51403
51487
51988
52005
52106
53202
53603
53915
53954
55028
56955
57086
57205
57298
57729
57997
60287
61613
62127
62745
63741
64143
64481
65198
65494
66646
67569
69106
70434
70891
72268
74910
77482
78516
78603
78884
79385
79813
79828
79934
80203
81485
82187
82236
82502
82729
82908
83218
83266
85001
86699
87391
87740
87834
88870
89556
89894
90255
91199
93340
93795
93835
94187
95267
95507
95534
95702
96246
96283
97118
97634
97640
98285
98512
98575
98803
100014
100590
100833
101024
101396
101651
102338
102676
10

In [23]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

corpus = train['tokens']
for text in corpus:
    if type(text) == type(''):
        text = re.sub("\d+", "", text)
    else:
        text = ''
vectorizer2 = CountVectorizer(analyzer='word', ngram_range=(1,1), min_df=30, max_df=90)
X2 = vectorizer2.fit_transform(corpus)
X2.toarray()[:round(len_train), :].shape

(242548, 14500)

In [21]:
X_train = pd.DataFrame(X2.toarray()[:round(len_train), :], columns = np.arange(X2.toarray().shape[1]))
X_test = pd.DataFrame(X2.toarray()[round(len_train):, :], columns = np.arange(X2.toarray().shape[1]))

In [29]:
from sklearn.metrics import roc_auc_score

model = LogisticRegression()
model.fit(X_train, train[:round(len_train)]['DEF'])
answer = pd.DataFrame(np.zeros(round(len_train)), columns=['DEF'])
for i, text in enumerate(train[round(len_train):]):
    if type(row['tokens']) == type(''):
        answer['DEF'][i:i+1] = model.predict(X_test[i:i+1])
    else:
        answer['DEF'][i:i+1] = 0

/home/jupyter-admin/.local/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:444: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [31]:
roc_auc_score(answer[:len(train) - round(len_train)], train[round(len_train):]['DEF'])

0.3782921318666821

In [28]:
len(answer)

1000

In [32]:
roc_auc_score(pd.DataFrame(np.zeros(round(len_train)), columns=['DEF']), train[round(len_train):]['DEF'])

ValueError: Only one class present in y_true. ROC AUC score is not defined in that case.